# Benchmarking Tutorial

Work in progress!

In [1]:
from fairx.dataset import BaseDataClass, CustomDataClass

from fairx.models.preprocessing import CorrRemover
from fairx.models.postprocessing import ThresholdAlgorithm
from fairx.models.inprocessing import TabFairGAN, Decaf, FairDisco
from fairx.utils import add_dict_result

from fairx.metrics.synthetic import SyntheticEvaluation

import warnings
warnings.filterwarnings('ignore')

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


## Dataset Loading

In [2]:
dataset_name = 'Compass'
sensitive_attr = 'race_African-American'
attach_target = True

In [3]:
ori_data_module = BaseDataClass(dataset_name, sensitive_attr, attach_target = attach_target)

Data loading complete
Target attribute: two_year_recid


In [4]:
res_dict = {

    'Methods': [],
    'Precision': [],
    'Recall': [],
    'Accuracy': [],
    'F1 Score': [],
    'Auroc': [],
    'Demographic Parity Ratio': [],
    'Equalized Odd Ratio': [],
    'Alpha-Precision': [],
    'Beta-Recall': [],
    'Authenticity': []
}

## Pre-processing: Correlation Remover

In [5]:
corr = CorrRemover(ori_data_module, sensitive_attr)

In [6]:
repaired_dataset, result = corr.fit()

In [7]:
print(result)

{'Methods': 'Correlation Remover', 'Precision': 0.6443057722308893, 'Recall': 0.5543624161073826, 'Accuracy': 0.6464646464646465, 'F1 Score': 0.595959595959596, 'Auroc': 0.6413051651454671, 'Demographic Parity Ratio': 0.5849668964575796, 'Equalized Odd Ratio': 0.6283087210922261}


## Post-processing: Threshold Optimizer

In [8]:
threshold = ThresholdAlgorithm(ori_data_module)

In [9]:
res = threshold.fit()

In [10]:
print(res)

{'Methods': 'Threshold Optimizer', 'Precision': 0.6519083969465649, 'Recall': 0.5731543624161074, 'Accuracy': 0.6553030303030303, 'F1 Score': 0.6100000000000001, 'Auroc': 0.6507011382998297, 'Demographic Parity Ratio': 0.9871992585445898, 'Equalized Odd Ratio': 0.7550925999776675}


## In-processing: Decaf

In [11]:
decaf = Decaf(ori_data_module, batch_size = 256, n_iter = 10, generated_sample_size = 1000)

model = decaf.fit()

Training Started


100%|███████████████████████████████████████| 1000/1000 [04:24<00:00,  3.78it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
--------

Epoch 9: 100%|█████████████████████| 66/66 [00:04<00:00, 13.22it/s, loss=-0.145]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|█████████████████████| 66/66 [00:05<00:00, 12.94it/s, loss=-0.145]
Data loading complete
(1000, 14)
(1000, 14)
{'Alpha-Precision': 0.19940000000000013, 'Beta-Recall': 0.002066666666666772, 'Authenticity': 1.0}
Training Completed
Generated Sample Saved


## In-processing: TabFairGAN

In [12]:
under_prev = 0
y_desire = 1

In [13]:
tabfairgan = TabFairGAN(under_prev, y_desire)

tabfairgan.fit(ori_data_module, batch_size = 256, epochs = 5)

epoch 1
training for fairness
epoch 2
training for fairness
epoch 3
training for fairness
epoch 4
training for fairness
epoch 5
training for fairness
Training Complete!
Data loading complete
(5278, 14)
(5278, 14)
{'Alpha-Precision': 0.02936718453959819, 'Beta-Recall': 0.0009852216748769127, 'Authenticity': 1.0}
generated data saved!
